In [ ]:
import pandas as pd
import numpy as np
import json
import re
import requests
from urllib.parse import urlparse


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
train_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/covid data/data/model_train.csv")
test_df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/covid data/data/model_test.csv")

train_label=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/covid data/data/Constraint_Train.csv")
test_label=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/covid data/data/Constraint_Val.csv")
#val_df = pd.read_csv('../../data/Constraint_Val.csv')
#test_df = pd.read_csv('../../data/english_test_with_labels.csv')
train_df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,model1_fake,model1_real,model2_fake,model2_real
0,0.286892,0.713108,0.604537,0.395463
1,0.093044,0.906956,0.000000,1.000000
2,0.857609,0.142391,0.894356,0.105644
3,0.098641,0.901359,0.000000,1.000000
4,0.116537,0.883463,0.000000,1.000000


In [ ]:
#train_feature_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/covid data/data/Train.csv')
#train_feature_df.head()


In [ ]:
#def get_df(df, feature_df, mode=''):
 # if mode in ['train', 'val']:
 #else:
  #  df = df.merge(feature_df, on=['id'], how='left')
  #df = df.iloc[:, :-2]
  #return df

In [ ]:
#train_df = get_df(train_feature_df,train_df, 'train')
#val_df = get_df(val_df, val_feature_df, 'val')
#test_df = get_df(test_df, test_feature_df, 'test')


In [ ]:
train_df=train_label.merge(train_df,how='outer',left_index=True,right_index=True)

In [ ]:
train_df.head()

,id,tweet,label,model1_fake,model1_real,model2_fake,model2_real
0,1,The CDC currently reports 99031 deaths. In gen...,real,0.286892,0.713108,0.604537,0.395463
1,2,States reported 1121 deaths a small rise from ...,real,0.093044,0.906956,0.000000,1.000000
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake,0.857609,0.142391,0.894356,0.105644
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real,0.098641,0.901359,0.000000,1.000000
4,5,Populous states can generate large case counts...,real,0.116537,0.883463,0.000000,1.000000


In [ ]:
test_df=test_label.merge(test_df,how='outer',left_index=True,right_index=True)

In [ ]:
username_dic = json.load(open("/content/drive/MyDrive/Colab Notebooks/covid data/data/train_prob_vectors.json"))
domain_dic = json.load(open("/content/drive/MyDrive/Colab Notebooks/covid data/data/train_prob_vectors_domain.json"))

In [ ]:
list(username_dic.items())

[('MoHFW_INDIA',
  {'fake_probability': 0.037037037037037035,
   'real_probability': 0.9629629629629629,
   'total_mentions': 162}),
 ('DrTedros',
  {'fake_probability': 0.0, 'real_probability': 1.0, 'total_mentions': 110}),
 ('ICMRDELHI',
  {'fake_probability': 0.009708737864077669,
   'real_probability': 0.9902912621359223,
   'total_mentions': 103}),
 ('drharshvardhan',
  {'fake_probability': 0.030927835051546393,
   'real_probability': 0.9690721649484536,
   'total_mentions': 97}),
 ('PIB_India',
  {'fake_probability': 0.0, 'real_probability': 1.0, 'total_mentions': 83}),
 ('alexismadrigal',
  {'fake_probability': 0.0, 'real_probability': 1.0, 'total_mentions': 47}),
 ('PMOIndia',
  {'fake_probability': 0.18181818181818182,
   'real_probability': 0.8181818181818182,
   'total_mentions': 44}),
 ('CDCMMWR',
  {'fake_probability': 0.0, 'real_probability': 1.0, 'total_mentions': 34}),
 ('DDNewslive',
  {'fake_probability': 0.0, 'real_probability': 1.0, 'total_mentions': 32}),
 ('Ashwin

In [ ]:
def username_feature(row, index):
  tweet = row["tweet"]
  print(tweet)
  lis = re.findall("@([a-z0-9_]+)", tweet, re.I)
  print(lis)
  val=0
  if len(lis)>0:
    for item in lis:
      try:
        val+=username_dic[item][index]
      except KeyError:
        val+=0
    val/=len(lis)  
  return val


def domain_feature(row, domain_dic, index):
  tweet=row["tweet"]
  try:
    shorturl = re.search("(?P<url>https?://[^\s]+)", tweet).group("url")
    r = requests.get(shorturl, timeout=180)
    expanded_url = r.url 
    # temp_domain = expanded_url.split('/')[2]
    domain = urlparse(expanded_url).netloc
    return domain_dic[domain][index]
  except Exception as e:
    domain=''
    return 0.0


In [ ]:
train_df["username_real"] = train_df.apply(lambda x: username_feature(x, "real_probability"), 1)
train_df["username_fake"] = train_df.apply(lambda x: username_feature(x, "fake_probability"), 1)

train_df["domain_real"] = train_df.apply(lambda x: domain_feature(x, domain_dic, "real_probability"), 1)
train_df["domain_fake"] = train_df.apply(lambda x: domain_feature(x, domain_dic, "fake_probability"), 1)


Streaming output truncated to the last 5000 lines.
Another 11 people who tested positive for #coronavirus have died in hospital in England bringing the total confirmed deaths in hospitals to 29687 NHS England says. Follow this story live 👇 https://t.co/NyZLPJyfxM
[]
288 new cases of #COVID19Nigeria; Lagos-88 Kwara-33 Osun-27 FCT-25 Enugu-25 Abia-20 Kaduna-17 Plateau-13 Rivers-13 Delta-10 Gombe-8 Ogun-4 Oyo-3 Katsina-1 Bauchi-1 44129 confirmed 20663 discharged 896 deaths https://t.co/NbSo7H63Lx
[]
With today’s two new cases and eight additional recovered cases our total number of active cases is 108. Of those 38 are imported cases in MIQ facilities and 70 are community cases.
[]
A post about the doctor Marcos Vargas of the Santojanni Hospital died from treating patients with coronavirus.
[]
REACT study finds that the prevalence of SARS-CoV-2 in England is increasing. https://t.co/EQ72HMQ39B #coronavirus https://t.co/6KBV4vMqEo
[]
Queens Man Admits to Being Non-Essential Worker https://t

KeyboardInterrupt: ignored

In [ ]:
print(train_df['username_real'])

0       0
1       0
2       0
3       0
4       0
       ..
6415    0
6416    0
6417    0
6418    0
6419    0
Name: username_real, Length: 6420, dtype: int64


In [ ]:
# val_df["username_real"] = val_df.apply(lambda x: username_feature(x, "real_probability"), 1)
# val_df["username_fake"] = val_df.apply(lambda x: username_feature(x, "fake_probability"), 1)

# val_df["domain_real"] = val_df.apply(lambda x: domain_feature(x, domain_dic, "real_probability"), 1)
# val_df["domain_fake"] = val_df.apply(lambda x: domain_feature(x, domain_dic, "fake_probability"), 1)


In [ ]:
#test_df["username_real"] = test_df.apply(lambda x: username_feature(x, "real_probability"), 1)
#test_df["username_fake"] = test_df.apply(lambda x: username_feature(x, "fake_probability"), 1)

#test_df["domain_real"] = test_df.apply(lambda x: domain_feature(x, domain_dic, "real_probability"), 1)
#test_df["domain_fake"] = test_df.apply(lambda x: domain_feature(x, domain_dic, "fake_probability"), 1)


In [ ]:
# domain_dic

In [ ]:
# pd.options.display.max_rows=None
train_df[train_df["domain_fake"]!=0.0]
train_df.head()

,id,tweet,label,model1_fake,model1_real,model2_fake,model2_real,username_real,username_fake,domain_real,domain_fake
0,1,cdc currently reports 99031 deaths general dis...,real,0.286892,0.713108,0.604537,0.395463,0,0,0.0,0.0
1,2,states reported 1121 deaths small rise last tu...,real,0.093044,0.906956,0.000000,1.000000,0,0,0.0,0.0
2,3,politically correct woman almost uses pandemic...,fake,0.857609,0.142391,0.894356,0.105644,0,0,0.0,0.0
3,4,indiafightscorona 1524 covid testing laborator...,real,0.098641,0.901359,0.000000,1.000000,0,0,0.0,0.0
4,5,populous states generate large case counts loo...,real,0.116537,0.883463,0.000000,1.000000,0,0,0.0,0.0


In [ ]:
train_df.to_csv('/content/drive/MyDrive/Colab Notebooks/covid data/data/final_prob.csv', index=False)